### data setup

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import differential_evolution as de

In [6]:
raw_data = pd.read_csv('dados\MC carpel H 5-min.csv')
raw_data.head(5)

,time,temperature,tga,dtga,dtga_5
0,0,450.35,10.32687,-0.000092,-0.00046
1,5,450.85,10.32643,-0.000088,-0.00044
2,10,451.25,10.32595,-0.000096,-0.00048
3,15,451.65,10.32550,-0.000090,-0.00045
4,20,452.15,10.32502,-0.000096,-0.00048


In [5]:
raw_data.dtypes

time             int64
temperature    float64
tga            float64
dtga           float64
dtga_5         float64
dtype: object

### functions

In [58]:
def time_interval_decorator(function):
    import time
    def wrapper(*args):
        initial_time = time.time()
        function(*args)
        final_time = time.time()
        elapsed_time = final_time - initial_time
        print(f'Time elapsed: {elapsed_time} seconds.')
    return wrapper

def theorical_temperature(**kwargs):
    return kwargs['initial_temperature'] + kwargs['heating_rate']/60 * kwargs['time']

def conversion(**kwargs):
    return (kwargs['initial_mass'] - kwargs['instant_mass']) / (kwargs['initial_mass'] - kwargs['final_mass'])

def conversion_rate(order, **kwargs):
    return (1 - conversion(**kwargs))**order

def arrhenius(pre_exp_factor, activation_energy, order, **kwargs):
    rate = pre_exp_factor * np.exp( -activation_energy / (kwargs['gas_constant'] * theorical_temperature(**kwargs)))
    rate *= conversion_rate(order, **kwargs)
    return rate

def rpi_model_for_mass(volatile_mass_fraction, pre_exp_factor, activation_energy, order, **kwargs):
    model = (kwargs['initial_mass'] - kwargs['final_mass'])
    model *= -volatile_mass_fraction * arrhenius(pre_exp_factor, activation_energy, order, **kwargs)
    return model

def objective_function(*args, **kwargs):
    return (rpi_model_for_mass(*args, **kwargs) - kwargs['real_data'])**2


### Guessing initial parameter intervals and hyper parameters for DE

In [63]:
# Bounds follows: [(component_fraction), (pre_exp_factor), (activation_energy), (order)]
hemicellulosis = [(0, 0.4), (0, 1e10), (0, 1e10), (0, 2.0)]
cellulosis = [(0, 0.5), (0, 1e10), (0, 1e10), (0, 2.0)]
lignin = [(0, 0.3), (0, 1e10), (0, 1e10), (0, 4.0)]
bounds = hemicellulosis + cellulosis + lignin

# DE hyper-parameters
strategy = 'best1bin'
maxiter = 1000
popsize = 15
tol = 1e-2
mutation = (0.5, 1)
recombination = 0.7
atol = 0

### Main script

In [57]:
# duplicating the data for safeguarding
data = raw_data.drop('dtga_5', axis=1).copy()

# Constants
initial_mass = data.tga.iloc[0] # mg
final_mass = data.tga.iloc[-1] # mg 
initial_temperature = data.temperature.iloc[0] # K
real_data = data.dtga # mg/s
gas_constant = 8.314 # J/mol*K
heating_rate = 5 # K/min

# conversion of mass over time
data['conversion'] = conversion(initial_mass=initial_mass, 
                                final_mass=final_mass, 
                                instant_mass=data.tga)

# calculating theoretical temperature
data['theoretical_temperature'] = theorical_temperature(initial_temperature=initial_temperature,
                                                        heating_rate=heating_rate,
                                                        time=data.time)

data.head(5)

,time,temperature,tga,dtga,conversion,theoretical_temperature
0,0,450.35,10.32687,-0.000092,0.000000,450.350000
1,5,450.85,10.32643,-0.000088,0.000066,450.766667
2,10,451.25,10.32595,-0.000096,0.000137,451.183333
3,15,451.65,10.32550,-0.000090,0.000205,451.600000
4,20,452.15,10.32502,-0.000096,0.000276,452.016667


In [64]:
@time_interval_decorator
def main():
    results = de(objective_function, bounds)
    params = results.x
    opt_fun = results.fun
    print(f'Params: {params}\nResult: {opt_fun}')